In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

 99% 327M/331M [00:09<00:00, 36.7MB/s]
100% 331M/331M [00:09<00:00, 38.0MB/s]


In [3]:
import zipfile
zip_ref=zipfile.ZipFile('/content/utkface-new.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [6]:
age=[]
gender=[]
img_path=[]


for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
df=pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [8]:
df.head()

,age,gender,img
0,29,0,29_0_3_20170117180817266.jpg.chip.jpg
1,75,0,75_0_0_20170116233015609.jpg.chip.jpg
2,69,1,69_1_3_20170109150308256.jpg.chip.jpg
3,3,0,3_0_2_20161219142321097.jpg.chip.jpg
4,34,0,34_0_1_20170113141042588.jpg.chip.jpg


In [9]:
#split the data into train and test

train_df=df.sample(frac=1,random_state=0).iloc[:20000]
test_df=df.sample(frac=1,random_state=0).iloc[20000:]

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,rotation_range =30,horizontal_flip=True,  vertical_flip=True,
                                   width_shift_range=0.2,height_shift_range = 0.2, zoom_range=0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
train_gen = train_datagen.flow_from_dataframe(train_df, directory =folder_path, x_col = 'img', y_col = ['age', 'gender'],
                                              target_size=(200,200), class_mode ='multi_output')

test_gen = test_datagen.flow_from_dataframe(test_df, directory =folder_path, x_col = 'img', y_col = ['age', 'gender'],
                                              target_size=(200,200), class_mode ='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [30]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.applications.vgg16 import VGG16 # Transfer Learning - VGG16
from keras.models import Model # Functional API

In [33]:
vgg_model = VGG16(include_top = False, input_shape = (200,200, 3))

input_5 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [24]:
vgg_model = VGG16(include_top = False, input_shape = (200,200, 3))

vgg_model.trainable = False

output = vgg_model.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten) # age hidden layer 1
dense2 = Dense(512, activation ='relu')(flatten) # gender hidden layer 1

dense3 = Dense(512, activation ='relu')(dense1) # age hidden layer 2
dense4 = Dense(512, activation = 'relu')(dense2) # gender hidden layer 2

output1 = Dense(1, activation='linear', name ='age')(dense3) # age output
output2 = Dense(1, activation='sigmoid', name ='gender')(dense4) # gender output


In [25]:
model = Model(inputs = vgg_model.input, outputs = [output1, output2])

In [26]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 200, 200, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 200, 200, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

In [27]:
import sklearn
from sklearn import metrics

In [28]:
model.compile(optimizer ='adam', loss = {'age' :'mae', 'gender' :'binary_crossentropy'},
              metrics = {'age': 'mae', 'gender':'accuracy'}, loss_weights = {'age':1, 'gender':99})

In [29]:
model.fit(train_gen, batch_size=32, epochs=10, validation_data = test_gen)

Epoch 1/10
625/625 [==============================] - 262s 396ms/step - loss: 68.4023 - age_loss: 11.4897 - gender_loss: 0.5749 - age_mae: 11.4897 - gender_accuracy: 0.7010 - val_loss: 52.7613 - val_age_loss: 9.5595 - val_gender_loss: 0.4364 - val_age_mae: 9.5595 - val_gender_accuracy: 0.7918
Epoch 2/10
242/625 [==========>...................] - ETA: 2:19 - loss: 59.8360 - age_loss: 10.0750 - gender_loss: 0.5026 - age_mae: 10.0750 - gender_accuracy: 0.7549

KeyboardInterrupt: ignored

In [ ]:
vgg_model = VGG16(include_top = False, input_shape = (200,200, 3))

In [ ]:
# Fine Tuning Method
vgg_model.trainable = True

set_trainable = False

for layer in vgg_model.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable :
    layer.trainable = True
  else:
    layer.trainable = False

for layer in vgg_model.layers:
  print(layer.name, layer.trainable)
